# The Double Slit Experiment - FEniCSx

This notebook demonstrates the simulation of the classic double slit experiment using the finite element method with FEniCSx. We solve the Helmholtz equation to model acoustic wave propagation through a domain with two narrow slits.

## Problem Description

The double slit experiment involves:
- An acoustic wave source at the inlet boundary
- A barrier with two narrow slits
- Wave propagation and interference patterns in the domain
- Absorbing boundary conditions to prevent reflections

We solve the time-harmonic Helmholtz equation:
$$\nabla^2 p + k^2 p = 0$$

where $p$ is the acoustic pressure and $k = \omega/c$ is the wave number.

In [ ]:
from mpi4py import MPI
from dolfinx import (
    fem,
    io,
    __version__ as dolfinx_version,
)
from dolfinx.fem.petsc import LinearProblem
import ufl
import numpy as np
import numpy.typing as npt
from packaging.version import Version


v_bc_wall_tag = 31
v_bc_inlet_tag = 29
v_bc_outlet_tag = 30
mesh_data = io.gmshio.read_from_msh("double_slit_mesh.msh", MPI.COMM_WORLD, 0, gdim=2)
if Version(dolfinx_version) > Version("0.9.0"):
    domain = mesh_data.mesh
    assert mesh_data.facet_tags is not None
    facet_tags = mesh_data.facet_tags
else:
    domain, boundary_tags, facet_tags = mesh_data

domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)

Info    : Reading 'double_slit_mesh.msh'...
Info    : 33 entities
Info    : 45295 nodes
Info    : 90590 elements
Info    : Done reading 'double_slit_mesh.msh'


array([], dtype=int32)

## Mesh and Boundary Conditions

The simulation domain is loaded from a GMSH mesh file (`double_slit_mesh.msh`) that contains:

- **Inlet boundary** (tag 29): Where the acoustic wave is generated
- **Outlet boundary** (tag 30): Where waves exit the domain  
- **Wall boundaries** (tag 31): Solid barriers including the double slit structure

The mesh includes facet tags that identify different boundary regions, allowing us to apply appropriate boundary conditions to each region.

The code handles different versions of DOLFINx to ensure compatibility across versions.

## Function Space and Physical Parameters

We define the function space for our unknown pressure field $p$ and specify the physical parameters for the acoustic wave propagation.

In [461]:

V = fem.functionspace(domain, ("Lagrange", 4))

# Air parameters
omega = 6 * np.pi * 1000  # Angular frequency (rad/s)
rho0 = 1.225  # kg/m^3
c = 340  # m/s
k = omega / c  # Wave number (1/m)
v_n = np.exp(1j * omega * 0)  # Normal velocity amplitude at inlet (m/s)

### Function Space
- **Lagrange elements of order 4**: Higher-order elements provide better accuracy for wave propagation problems
- **Complex-valued**: Required for time-harmonic analysis with phase information

### Physical Parameters
- **Angular frequency (ω)**: 6π × 1000 rad/s ≈ 3 kHz
- **Air density (ρ₀)**: 1.225 kg/m³ (standard air density)
- **Sound speed (c)**: 340 m/s (speed of sound in air)
- **Wave number (k)**: k = ω/c, determines the wavelength
- **Inlet velocity (v_n)**: Complex amplitude of normal velocity at the source

## Integration Measures

We define the integration measure for boundary integrals using UFL (Unified Form Language).

In [462]:
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

The `ds` measure allows us to integrate over specific boundary segments using the facet tags:
- `ds(v_bc_inlet_tag)`: Integration over the inlet boundary
- `ds(v_bc_outlet_tag)`: Integration over the outlet boundary  
- `ds(v_bc_wall_tag)`: Integration over wall boundaries

## Variational Formulation

We formulate the weak form of the Helmholtz equation with appropriate boundary conditions for the double slit problem.

In [463]:
p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = (
    ufl.inner(ufl.grad(p), ufl.grad(v)) * ufl.dx
    # + 1j * rho0 * omega / Z * ufl.inner(p, v) * ds(v_bc_inlet_tag)
    - (k**2)*p*ufl.conj(v) * ufl.dx
- 1j*k * p * ufl.conj(v) * ds
- k**2 * ufl.inner(p, v) * ufl.dx
)
L = -1j * omega * rho0 * ufl.inner(v_n, v) *ds(v_bc_inlet_tag)

### Bilinear Form (a)
The left-hand side contains:
1. **∇p · ∇v dx**: Laplacian term from the Helmholtz equation
2. **-k²p·v̄ dx**: Wave number term (using complex conjugate of test function)
3. **-ik·p·v̄ ds**: Absorbing boundary condition to prevent reflections

### Linear Form (L)  
The right-hand side represents:
- **Source term**: -iωρ₀v_n applied at the inlet boundary
- This creates the incident acoustic wave

The formulation uses complex arithmetic to handle the time-harmonic nature of the problem.

## Boundary Conditions and Solution

We apply Dirichlet boundary conditions at the inlet and solve the linear system using PETSc.

In [ ]:
bc_inlet = fem.dirichletbc(v_n, fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_inlet_tag)), V)

### Inlet Boundary Condition
- **Dirichlet BC**: Prescribes the pressure amplitude at the inlet
- **DOF location**: Uses topological information to find degrees of freedom on the inlet boundary
- This ensures a controlled wave source at the inlet

In [465]:
p_a = fem.Function(V)
p_a.name = "pressure"

problem = LinearProblem(
    a,
    L,
    u=p_a,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        # "pc_factor_mat_solver_type": "mumps",
    },
    bcs=[bc_inlet],
)

### Linear Problem Setup
- **Solution function**: `p_a` stores the computed pressure field
- **PETSc options**: 
  - `"ksp_type": "preonly"`: Direct solver (no iterative method)
  - `"pc_type": "lu"`: LU decomposition for direct factorization
  - Commented MUMPS option for parallel direct solving
- **Boundary conditions**: Applied at the inlet

In [466]:

problem.solve()
p_a.x.scatter_forward()

### Solving the System
1. **solve()**: Assembles and solves the linear system Ax = b
2. **scatter_forward()**: Ensures proper distribution of solution data across MPI processes

The solution `p_a` now contains the complex pressure field showing wave propagation and interference patterns through the double slit.

In [468]:
with io.VTKFile(domain.comm, "solution.pvd", "w") as file:
    file.write_mesh(domain)
    file.write_function([p_a])

## Output and Visualization

The solution is written to a ParaView format file (`solution.pvd`) for visualization:

- **Mesh geometry**: Domain structure including the double slit barrier
- **Pressure field**: Complex-valued solution showing wave amplitude and phase
- **Visualization**: Can be opened in ParaView to see:
  - Wave propagation from inlet
  - Diffraction through the slits  
  - Interference patterns downstream
  - Wave amplitude and phase distributions

The results demonstrate the classic double slit interference pattern with constructive and destructive interference regions downstream of the slits.